# Methylation Evaluation

The [minfi](https://htmlpreview.github.io/?https://github.com/hansenlab/tutorial.450k/blob/master/inst/doc/methylation450k.html) package is commonly used for methylation array analysis. Normalization is handled by the included subset-quantile within array normalization (SWAN) method. SWAN normalizes methylation array data by selecting a subset of probes from both types (Infinium I and Infinium II).

The SWAN publication describes this method thusly:

> The SWAN method has two parts. First, an average quantile distribution is determined using a subset of probes defined to be biologically similar based on CpG content. This is achieved by randomly selecting N Infinium I and II probes that have one, two and three underlying CpGs, where N is the minimum number of probes in the six sets of Infinium I and II probes with one, two and three probe body CpGs. If no probes have been filtered out - for example, sex chromosome probes, and so on - N = 11,303. This results in a pool of 3N Infinium I and 3N Infinium II probes. Due to the vast differences in their distributions (Figure 2), the subsequent processing is performed independently on both the methylated and unmethylated channels. The subset for each probe type, from each channel (methylated or unmethylated), is sorted by increasing intensity. The value of each of the 3N pairs of observations is then assigned to be the mean intensity of the two probe types for that row or 'quantile'. This is the standard quantile procedure. The second step is to then adjust the intensities of the remaining probes, of which there are many more Infinium II than I, by interpolation onto the distribution of the subset probes. This is done for each probe type separately using linear interpolation between the subset probes to define the new intensities. Consequently, while the distribution of the subset is identical, the intensity distribution of Infinium I probes is still vastly different from the distribution of Infinium II probes (Figure S2 in Additional file 1).

This description suggests that individual samples can be normalized separately since the distribution of each channel within a sample is adjusted based on the subset of probes selected. The only difference between independent runs of SWAN is the selection of the subset of probes. This can be controlled by setting a seed in R (`set.seed(N)`).


# Setup
This evaluation requires a number of packages to be installed in the environment. To accomplish this, I have used [conda](https://conda.io/projects/conda/en/latest/user-guide/install/index.html) along with the `bioconda`, `r`, and `conda-forge` channels. 

In [1]:
%%bash
# Uncomment the lines below to create the conda environment.
# - I'm not necessarily advocating you do this in this notebook, 
#   probably want to set it up in a terminal and then start this Jupyter notebook after :).

# conda create -n methylation \
#              -c anaconda \
#              -c bioconda \
#              -c conda-forge \
#              numpy \
#              pandas \
#              r-dplyr \
#              bioconductor-minfi \
#              bioconductor-illuminahumanmethylationepicmanifest \
#              bioconductor-illuminahumanmethylationepicanno.ilm10b4.hg19 \
#              
#              -y
# conda init bash
# source ~/.bashrc
# conda activate methylation

# Analysis

In [2]:
import pandas as pd
import numpy as np

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
R.version.string

[1] "R version 4.2.2 (2022-10-31)"


In [5]:
%%R
#Load libraries
library(dplyr)
library(minfi)
library(IlluminaHumanMethylationEPICmanifest)
library(IlluminaHumanMethylationEPICanno.ilm10b4.hg19)


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: Loading required package: BiocGenerics

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sap

For reproducibility, we'll fix the seed value.

In [6]:
%%R
set.seed(1)

In [7]:
%%R
RGSet <- read.metharray("~/data/COMET/methylation_data/201533640034_R01C01", verbose = TRUE, force = TRUE)

R[write to console]: [read.metharray] Reading 201533640034_R01C01_Grn.idat

R[write to console]: [read.metharray] Reading 201533640034_R01C01_Red.idat

R[write to console]: [read.metharray] Read idat files in 0.5 seconds

R[write to console]: [read.metharray] Creating data matrices ... 
R[write to console]: done in 1.4 seconds

R[write to console]: [read.metharray] Instantiating final object ... 
R[write to console]: done in 0.1 seconds



In [8]:
%%R
manifest <- getManifest(RGSet)
manifest

IlluminaMethylationManifest object
Annotation
  array: IlluminaHumanMethylationEPIC
Number of type I probes: 142262 
Number of type II probes: 724574 
Number of control probes: 635 
Number of SNP type I probes: 21 
Number of SNP type II probes: 38 


In [9]:
%%R
# Load raw data into a MethylSet object be converting red/green
# channels into a matrix of methlyated and unmethylated signals
MSet <- preprocessRaw(RGSet)

In [10]:
%%R
# Convert to a RatioSet
RSet <- ratioConvert(MSet, what = "both", keepCN = TRUE)

In [11]:
%%R
# Add genomic coordinates to each probe (plus additional annotation)
GRset <- mapToGenome(RSet)

In [12]:
%%R
# Take the genomic mapped RatioSet and fill Beta values (non-normalized).
# Get the NON-normalized beta values:
beta <- getBeta(GRset)

In [13]:
%%R
# Get M-value matrix and copy-number matrix
M <- getM(GRset)
CN <- getCN(GRset)

In [14]:
%%R
# Get sample names
sampleNames <- sampleNames(GRset)
sampleNames

[1] "201533640034_R01C01"


In [15]:
%%R
# Get probe names
featureNames <- featureNames(GRset)
featureNames

    [1] "cg14817997"      "cg26928153"      "cg16269199"      "cg13869341"     
    [5] "cg14008030"      "cg12045430"      "cg20826792"      "cg20253340"     
    [9] "cg02404219"      "cg21870274"      "cg03130891"      "cg04098293"     
   [13] "cg16382250"      "cg24335620"      "cg22463138"      "cg16162899"     
   [17] "cg01803908"      "cg17149495"      "cg22802167"      "cg24669183"     
   [21] "cg17308840"      "cg17866181"      "cg25018832"      "cg26679879"     
   [25] "cg22519184"      "cg11894324"      "cg24159721"      "cg02288058"     
   [29] "cg15174812"      "cg12376350"      "cg08477687"      "cg00034556"     
   [33] "cg00645010"      "cg15394630"      "cg05001044"      "cg03344490"     
   [37] "cg17501828"      "cg11330075"      "cg23917638"      "cg18024856"     
   [41] "cg18761878"      "cg27534567"      "cg08858441"      "cg03348902"     
   [45] "cg01070250"      "cg21996134"      "cg07264491"      "cg01577251"     
   [49] "cg13128577"      "cg15560884"  

  [409] "cg08067365"      "cg20062691"      "cg03811829"      "cg11211792"     
  [413] "cg04788999"      "cg16526047"      "cg07123796"      "cg26560020"     
  [417] "cg09363892"      "cg03185752"      "cg15153791"      "cg04336684"     
  [421] "cg22482746"      "cg25177113"      "cg08223393"      "cg07008945"     
  [425] "cg11726631"      "cg19813362"      "cg00299257"      "cg10046671"     
  [429] "cg24704476"      "cg13917577"      "cg17801765"      "cg13100600"     
  [433] "cg18272461"      "cg07787977"      "cg02632261"      "cg07396477"     
  [437] "cg15574137"      "cg25774148"      "cg17021880"      "cg02288964"     
  [441] "cg02341264"      "cg06712559"      "cg17928920"      "cg14663984"     
  [445] "cg16318112"      "cg09248054"      "cg15840462"      "cg24598973"     
  [449] "cg16502826"      "cg18147233"      "cg20741620"      "cg00525597"     
  [453] "cg26007962"      "cg27541454"      "cg25759916"      "cg01150641"     
  [457] "cg23625715"      "cg26222311"  

  [817] "cg19237691"      "cg08641866"      "cg14886269"      "cg15706223"     
  [821] "cg06097659"      "cg04343794"      "cg07671976"      "cg25725823"     
  [825] "cg00086243"      "cg10583942"      "cg07119157"      "cg19092396"     
  [829] "cg12701018"      "cg25144135"      "cg20996124"      "cg16266669"     
  [833] "cg02296931"      "cg15924570"      "cg15993652"      "cg25625296"     
  [837] "cg09503311"      "cg20544186"      "cg01437515"      "cg12759096"     
  [841] "cg22335801"      "cg14661234"      "cg23166857"      "cg09586191"     
  [845] "cg01341040"      "cg21616720"      "cg21815220"      "cg17084044"     
  [849] "cg16252905"      "cg26312965"      "cg09315878"      "cg01075305"     
  [853] "cg03058163"      "cg03580801"      "cg19862782"      "cg12219387"     
  [857] "cg11727921"      "cg01783406"      "cg14531564"      "cg09276842"     
  [861] "cg07548512"      "cg22798412"      "cg27477059"      "cg25578497"     
  [865] "cg00656410"      "cg10206039"  

 [1225] "cg15492552"      "cg05572157"      "cg18534079"      "cg00046914"     
 [1229] "cg02416993"      "cg07726485"      "cg23377249"      "cg24356704"     
 [1233] "cg27503832"      "cg22643873"      "cg05397857"      "cg05387269"     
 [1237] "cg16342853"      "cg14992101"      "cg09996760"      "cg09864992"     
 [1241] "cg14094639"      "cg06203720"      "cg01508988"      "cg07395004"     
 [1245] "cg19871494"      "cg09741074"      "cg08433720"      "cg21513204"     
 [1249] "cg15604233"      "cg11915315"      "cg20048602"      "cg20439722"     
 [1253] "cg19823490"      "cg21478504"      "cg04615596"      "cg05364533"     
 [1257] "cg15617543"      "cg18546249"      "cg11552267"      "cg05589276"     
 [1261] "cg02144516"      "cg20435334"      "cg26865478"      "cg16361266"     
 [1265] "cg20434150"      "cg08253842"      "cg14224772"      "cg03322002"     
 [1269] "cg08109545"      "cg12033178"      "cg08158776"      "cg04880874"     
 [1273] "cg15588177"      "cg22232737"  

 [1633] "cg05934012"      "cg07479014"      "cg09126090"      "cg16358446"     
 [1637] "cg08289140"      "cg11521998"      "cg09172260"      "cg15158067"     
 [1641] "cg21240031"      "cg26680206"      "cg15034659"      "cg11649182"     
 [1645] "cg25311963"      "cg07147033"      "cg13897348"      "cg19246761"     
 [1649] "cg01229567"      "cg15817635"      "cg15652741"      "cg10705488"     
 [1653] "cg11140595"      "cg06761793"      "cg18123911"      "cg17271960"     
 [1657] "cg25004270"      "cg25880288"      "cg08912129"      "cg25164153"     
 [1661] "cg26169249"      "cg07807975"      "cg25010770"      "cg10756715"     
 [1665] "cg05941212"      "cg13947830"      "cg26295592"      "cg00781053"     
 [1669] "cg23605175"      "cg15369276"      "cg25459752"      "cg15558634"     
 [1673] "cg10970349"      "cg15323084"      "cg19165249"      "cg09082814"     
 [1677] "cg23957653"      "cg09061216"      "cg03975652"      "cg25500853"     
 [1681] "cg04292080"      "cg17456675"  

...

[96033] "cg12191964"      "cg10940651"      "cg04483548"      "cg11676221"     
[96037] "cg03762820"      "cg27162030"      "cg02901722"      "cg21555271"     
[96041] "cg24445271"      "cg23222302"      "cg15461431"      "cg23330051"     
[96045] "cg08078339"      "cg06173543"      "cg15096488"      "cg02757223"     
[96049] "cg04300730"      "cg02881939"      "cg06449458"      "cg11362904"     
[96053] "cg24650394"      "cg16984812"      "cg00820919"      "cg11659317"     
[96057] "cg16864139"      "cg09067818"      "cg01605419"      "cg22475025"     
[96061] "cg21926465"      "cg12617935"      "cg18856652"      "cg15523958"     
[96065] "cg15754153"      "cg17110364"      "cg09325404"      "cg11267669"     
[96069] "cg14230606"      "cg09132607"      "cg02111748"      "cg18817560"     
[96073] "cg13324389"      "cg20563666"      "cg14454477"      "cg08726338"     
[96077] "cg26877938"      "cg21621660"      "cg21751085"      "cg05570153"     
[96081] "cg24235864"      "cg07064251"  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "cg01502208"     
[98661] "cg10854528"      "cg17376041"      "cg02463166"      "cg22654940"     
[98665] "cg10929758"      "cg05841263"      "cg08981537"      "cg21270675"     
[98669] "cg25407528"      "cg15706621"      "cg08556541"      "cg05276829"     
[98673] "cg23984849"      "cg03365520"      "cg22064227"      "cg15714776"     
[98677] "cg12949431"      "cg08701028"      "cg02158822"      "cg05656039"     
[98681] "cg25885402"      "cg23402524"      "cg03344672"      "cg26432715"     
[98685] "cg08600810"      "cg16665410"      "cg11003199"      "cg26456257"     
[98689] "cg06497591"      "cg00710366"      "cg26097391"      "cg13088660"     
[98693] "cg16920712"      "cg15779849"      "cg16198930"      "cg04678916"     
[98697] "cg20341504"      "cg14919082"      "cg11355849"      "cg23139792"     
[98701] "cg05317006"      "cg25811247"      "cg20454887"      "cg04181892"     
[98705] "cg25122629"      "cg20254293"      "cg01193335"      "cg14709837"     
[98709] "cg23576807" 

[99313] "cg02611899"      "cg06795523"      "cg18775824"      "cg00869533"     
[99317] "cg15923705"      "cg16288522"      "cg05133691"      "cg22140100"     
[99321] "cg11068356"      "cg02277376"      "cg07259271"      "cg18631471"     
[99325] "cg19051908"      "cg06281205"      "cg07282284"      "cg24302086"     
[99329] "cg18867141"      "cg15664967"      "cg09452692"      "cg19679281"     
[99333] "cg18816705"      "cg06968475"      "cg02845365"      "cg23500942"     
[99337] "cg21360335"      "cg26836809"      "cg24308989"      "cg19421528"     
[99341] "cg11018410"      "cg00470117"      "cg03607916"      "cg25724323"     
[99345] "cg09026213"      "cg00774457"      "cg24906015"      "cg05138064"     
[99349] "cg08397437"      "cg16117171"      "cg26451804"      "cg01125436"     
[99353] "cg10408627"      "cg26219680"      "cg17811563"      "cg16349527"     
[99357] "cg01102471"      "cg11022093"      "cg17357341"      "cg09700327"     
[99361] "cg11847608"      "cg25862458"  

[99721] "cg06815041"      "cg01910875"      "cg06861945"      "cg19263721"     
[99725] "cg13290274"      "cg11472573"      "cg13659415"      "cg12225766"     
[99729] "cg16704832"      "cg00386013"      "cg26888813"      "cg11723397"     
[99733] "cg23519930"      "cg07067189"      "cg17862961"      "cg08129754"     
[99737] "cg21173221"      "cg06688433"      "cg19275978"      "cg12180123"     
[99741] "cg10010189"      "cg03073791"      "cg18222526"      "cg21898392"     
[99745] "cg10136738"      "cg07750914"      "cg08385097"      "cg25962992"     
[99749] "cg18745274"      "cg08498300"      "cg01688574"      "cg15033424"     
[99753] "cg06395423"      "cg25136868"      "cg11124436"      "cg06350121"     
[99757] "cg25959798"      "cg27232910"      "cg09036712"      "cg19905379"     
[99761] "cg01240169"      "cg14337632"      "cg20489115"      "cg16562296"     
[99765] "cg14324153"      "cg04798790"      "cg12280024"      "cg08593873"     
[99769] "cg25133156"      "cg01845673"  

In [16]:
%%R
gr <- granges(GRset)
annotation <- getAnnotation(GRset)

In [17]:
%%R
set.seed(1)
# Perform SWAN normalization on beta values
GRset.swan_norm <- preprocessSWAN(RGSet)
beta_swan_norm <- getBeta(GRset.swan_norm)

In [18]:
%%R -o beta_swan_norm_genomic
# Write the normalized beta-values that have NOT yet had
# low-variance probes filtered out
# Filter to only those that are mappable to the genome.
RSet_genomic <- ratioConvert(GRset.swan_norm)
GRset_genomic <- mapToGenome(RSet_genomic)
beta_swan_norm_genomic <- getBeta(GRset_genomic)
colnames(beta_swan_norm_genomic) <- colnames(CN)

In [19]:
print(beta_swan_norm_genomic)
print(beta_swan_norm_genomic.shape)

[[0.8319394 ]
 [0.87825323]
 [0.74041298]
 ...
 [0.84366217]
 [0.5464304 ]
 [0.41737797]]
(865859, 1)


Read in data for a second sample.

In [20]:
%%R
second_RGSet <- read.metharray("~/data/COMET/methylation_data/201533640009_R08C01", verbose = TRUE, force = TRUE)

R[write to console]: [read.metharray] Reading 201533640009_R08C01_Grn.idat

R[write to console]: [read.metharray] Reading 201533640009_R08C01_Red.idat

R[write to console]: [read.metharray] Read idat files in 0.5 seconds

R[write to console]: [read.metharray] Creating data matrices ... 
R[write to console]: done in 0.9 seconds

R[write to console]: [read.metharray] Instantiating final object ... 
R[write to console]: done in 0.0 seconds



Now we'll combine the data for both samples so that we can analysis them together.

In [21]:
%%R
combined <- combine(RGSet, second_RGSet)

In [22]:
%%R -o combined_beta_swan_norm_genomic
set.seed(1)
combined_GRset <- mapToGenome(combined)
combined_CN <- getCN(combined_GRset)
combined_GRset.swan_norm <- preprocessSWAN(combined)
combined_RSet_genomic <- ratioConvert(combined_GRset.swan_norm)
combined_GRset_genomic <- mapToGenome(combined_RSet_genomic)
combined_beta_swan_norm_genomic <- getBeta(combined_GRset_genomic)
colnames(combined_beta_swan_norm_genomic) <- colnames(combined_CN)


In [23]:
print(combined_beta_swan_norm_genomic)

[[0.8319394  0.91933784]
 [0.87825323 0.86565846]
 [0.74041298 0.74228972]
 ...
 [0.84366217 0.81228846]
 [0.5464304  0.49253039]
 [0.41737797 0.17487666]]


In [24]:
sample_from_combined = combined_beta_swan_norm_genomic[:,0]
print(sample_from_combined)

[0.8319394  0.87825323 0.74041298 ... 0.84366217 0.5464304  0.41737797]


Now to check equality between the two runs of the normalization.

In [25]:
np.array_equal(beta_swan_norm_genomic, sample_from_combined)

False

The inequality is not a surprise. These are very long floating point numbers, so some difference is expected due to rounding. So we need to compare with a tolerance.

Numpy offers the `allclose` function which compares array values within a tolerance. However, it crashes this kernel, presumably, due to the size of the arrays.

In [26]:
#np.allclose(beta_swan_norm_genomic, sample_from_combined)

Instead we will iterate over the arrays and check manually. Then check to see that all values are within the allowable tolerance.

In [27]:
value = []
atol = 1e-8
rtol = 1e-5
for a,b in zip(beta_swan_norm_genomic, sample_from_combined):
    value.append(abs(a - b) <= (atol + rtol * abs(b)))

In [28]:
print(all(value))

True
